In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import scipy.optimize
import scipy.stats
import csv
from scipy.io import loadmat
from datetime import datetime, date, time, timedelta
sns.set_theme()

#import ipynb.fs.defs.analysis_fxns as af

In this file:
- importing demographic and dates of collection data from mat files 
- filling in menstruation/ovulation dates when necessary
- importing all data collected
- adding if day of data is a menstruation/ovulation date
- adding in days since menstruation and phase values 
- adding in data such as CGM change rate, filling in basal delivery on a 5 min scale
- merging CGM/insulin data with mens data
- getting a baseline basal rate by averaging basal delivery from 0:00 - 5:59, 6:00 - 11:59, 12:00 - 17:59, 18:00 - 23:59
- functions to return statistics about BGI, BG, and insulin delivery on hourly/daily scale
- generating blocks for both fine and coarse grained analysis and details of base construction
- sorting runs and patterns, removing duplicates and generating statistics for outcomes of each pattern
- time and phase split pattern analysis

# import mat files (dates), adding ovulation data where needed 

In [34]:
path = 'uva_data/dataCollDates.mat'
mat = scipy.io.loadmat(path)

In [35]:
data = [[row.flat[0] for row in line] for line in mat['dataCollDates']]
df_collDates = pd.DataFrame(data)
df_collDates.columns = df_collDates.iloc[0]
df_collDates = df_collDates[1:]
df_collDates = df_collDates.reset_index(drop = True)

In [36]:
start_strings = []
end_strings = []
length_of_coll = []
for i in range(len(df_collDates)):
    start_date = int(df_collDates.iloc[i]['Start Date'])
    start_str = (datetime.fromordinal(start_date) + timedelta(days=(start_date)%1) - timedelta(days = 366))
    start_strings.append(start_str)

    end_date = int(df_collDates.iloc[i]['End Date'])
    end_str = (datetime.fromordinal(end_date) + timedelta(days=(start_date)%1) - timedelta(days = 366))
    end_strings.append(end_str)

    length_of_coll.append(end_str-start_str)

In [ ]:
df_collDates['Start (dt)'] = start_strings
df_collDates['End (dt)'] = end_strings
df_collDates['Days collected'] = length_of_coll


In [7]:
print(df_collDates['Days collected'].mean())
print(df_collDates['Days collected'].std())

95 days 18:24:00
12 days 08:56:33.961339029


In [38]:
path = 'uva_data/subjData.mat'
mat = scipy.io.loadmat(path)

In [39]:
# formatting mat data into a dataframe
data = [[row for row in line] for line in mat['dates']]
df_dates = pd.DataFrame(data)

df_dates.columns = ['ID', 'Menses dates', 'Ovulation dates', 'Group']
df_dates = df_dates[1:]
df_dates = df_dates.reset_index(drop = True)

In [40]:
# these columns were arrays of arrays ( [[data]] ) - getting rid of the extra shell 
df_dates['ID'] = df_dates['ID'].str[0]
df_dates['Group'] = df_dates['Group'].str[0]

In [41]:
# making columns of strings for mens/ovulation dates from matlab hash
str_mens_dates = []
str_ov_dates = []
for i in range(len(df_dates)):
    menses = df_dates['Menses dates'][i]
    subj_str_mens = []
    
    for j in range(len(menses)):
        date = menses[j][0]
        dt = datetime.fromordinal(int(date)) + timedelta(days=int(date)%1) - timedelta(days = 366)
        subj_str_mens.append(dt.strftime('%Y-%m-%d'))
    str_mens_dates.append(subj_str_mens)
    
    ovs = df_dates['Ovulation dates'][i]
    subj_str_ovs = []
    
    for j in range(len(ovs)):
        date = ovs[j][0]
        dt = datetime.fromordinal(int(date)) + timedelta(days=int(date)%1) - timedelta(days = 366)
        subj_str_ovs.append(dt.strftime('%Y-%m-%d'))
    str_ov_dates.append(subj_str_ovs)

In [42]:
df_dates['Menses dates (str)'] = str_mens_dates
df_dates['Ovulation dates (str)'] = str_ov_dates

In [43]:
def fill_missing_ov_dates(df_dates):
    '''adds midpoint dates to cycles where no ovulation data was recorded'''
    copy_df = df_dates.copy() # was having issues with df returning correctly - temp solution
    
    inf_ov_dates_col = []
    all_ov_dates_col = []

    for i in range(len(copy_df)): # for each subject
        # get datetime format... 
        mens_dates_str = copy_df.iloc[i]['Menses dates (str)']
        mens_dates_dt = []
        for date in mens_dates_str:
            mens_dates_dt.append(datetime.strptime(date, '%Y-%m-%d'))

        orig_ov_dates_str = copy_df.iloc[i]['Ovulation dates (str)']
        orig_ov_dates_dt = []
        for date in orig_ov_dates_str:
            orig_ov_dates_dt.append(datetime.strptime(date, '%Y-%m-%d'))

        inf_ov_dates_dt = []
        cycle_lengths = []
        for j in range(len(mens_dates_dt)): # for each cycle, check if an ovulation date exists 
            has_ov = False
            d1 = mens_dates_dt[j]
            if j == len(mens_dates_dt) - 1:
                d2 = datetime.strptime('2023-12-31', '%Y-%m-%d') # set arbitrary end date if last mens - does not impact ov date assignment
            else:
                d2 = mens_dates_dt[j+1]
            for m in range(len(orig_ov_dates_dt)):
                if d1 <= orig_ov_dates_dt[m] <= d2: # if ov in between two mens, has ov = true
                    has_ov = True
            
            if has_ov == False: # if no ov found 
                if j != (len(mens_dates_dt) - 1): # if not last mens date, take midpoint of cycle as approx ovulation date
                    mid_date = mens_dates_dt[j] + ( (mens_dates_dt[j+1] - mens_dates_dt[j])/ 2 )
                    inf_ov_dates_dt.append(mid_date)
                    cycle_lengths.append(mens_dates_dt[j+1] - mens_dates_dt[j])

                elif len(mens_dates_dt) == 1: # for one weird fringe case with 1 ov date and 1 mens date :(
                    delta_t = mens_dates_dt[0] - orig_ov_dates_dt[0]
                    mid_date = mens_dates_dt[j] + delta_t
                    inf_ov_dates_dt.append(mid_date)

                else: # if last cycle, take avg cycle length and divide by 2 for approx ovulation date 
                    mean_cycle_length = pd.to_timedelta(pd.Series(cycle_lengths)).mean()
                    mid_date = mens_dates_dt[j] + (mean_cycle_length / 2)
                    inf_ov_dates_dt.append(mid_date)

            elif (has_ov == True) and (j!= len(mens_dates_dt) - 1): # adding cycle length to arr if ov found
                cycle_lengths.append(mens_dates_dt[j+1] - mens_dates_dt[j])
        
        all_ov_dates_dt = orig_ov_dates_dt
        all_ov_dates_dt.extend(inf_ov_dates_dt)
        all_ov_dates_dt.sort()

        all_ov_dates_str = []
        inf_ov_dates_str = []

        for d in all_ov_dates_dt:
            all_ov_dates_str.append(d.strftime('%Y-%m-%d'))
        for d in inf_ov_dates_dt:
            inf_ov_dates_str.append(d.strftime('%Y-%m-%d'))
        
        all_ov_dates_col.append(all_ov_dates_str)
        inf_ov_dates_col.append(inf_ov_dates_str)

    copy_df['inferred ovulation dates'] = inf_ov_dates_col
    copy_df['filled ovulation dates'] = all_ov_dates_col
    return copy_df

In [44]:
df_dates = fill_missing_ov_dates(df_dates)

# import not mat files

In [45]:
# get files 
data = glob.glob('uva_data/ind_data/*****_data.csv')

In [46]:
# make files into dataframes
dfs = []
for file in data:
    dfs.append(pd.read_csv(file, low_memory = False))

In [48]:
def add_mens_ov_dates(df, df_dates):
    '''adds bool columns for menses and ovulation dates'''
    # get subject id, use to get menses/ovulation dates 
    subj_id = df.iloc[0]['id']
    mens_dates = list(df_dates[df_dates['ID'] == subj_id]['Menses dates (str)'])[0]
    ov_dates =  list(df_dates[df_dates['ID'] == subj_id]['filled ovulation dates'])[0]
    
    # split t into date/time columns
    df['t'] = df['t'].astype(str)
    df[['date', 'time']] = df['t'].str.split(pat = ' ', expand = True)
    
    # makes new columns with bool value if date is in 
    df['is mens'] = np.where(df['date'].isin(mens_dates), 1, 0)
    df['is ov'] = np.where(df['date'].isin(ov_dates), 1, 0)
    
    # makes sure t is still in datetime format
    df['t'] = pd.to_datetime(df['t'])

In [49]:
for df in dfs:
    add_mens_ov_dates(df, df_dates)

In [50]:
def days_since_mens(df, df_dates):
    '''adding dates since mens column and phase column, phases are 1-4'''
    
    # getting mens/ov dates and all dates in list form
    subj_id = df.iloc[0]['id']
    mens_dates_t = list(df_dates[df_dates['ID'] == subj_id]['Menses dates (str)'])[0]
    ov_dates_t = list(df_dates[df_dates['ID'] == subj_id]['filled ovulation dates'])[0]
    dates = list(df['date'])

    days_since = np.zeros(len(df))
    phase_list = np.zeros(len(df))
    last_date = dates[0]
    before_first = True
    days_since_count = np.nan
    phase = np.nan # 1 for menstruation, 2 for follicular, 3 for ovulation, 4 for luteal. nan before first mens date?

    # creating buffer ranges - 1 day before and 1 day after date given for a total of 3 days
    mens_buffer_dates = []
    for day in mens_dates_t:
        mens_dt = datetime.strptime(day, '%Y-%m-%d')
        mens_buffer_dates.append((mens_dt - timedelta(days = 1)).strftime('%Y-%m-%d'))
        mens_buffer_dates.append((mens_dt + timedelta(days = 1)).strftime('%Y-%m-%d'))

    ov_buffer_dates = []
    for day in ov_dates_t:
        ov_dt = datetime.strptime(day, '%Y-%m-%d')
        ov_buffer_dates.append((ov_dt - timedelta(days = 1)).strftime('%Y-%m-%d'))
        ov_buffer_dates.append((ov_dt + timedelta(days = 1)).strftime('%Y-%m-%d'))

    for i in range(len(dates)):

        # TO GET COUNT
        # if date matches menses date, reset count 
        if dates[i] in mens_dates_t: # if mens date, reset the count
            before_first = False
            phase = 1
            days_since_count = 0

        if before_first == True: # if before the first menses date provided, set pos as nan
            days_since[i] = np.nan

        else:
            #if date is new, count + 1 and set last_date to date 
            if last_date != dates[i]:
                days_since_count += 1 
            last_date = dates[i]
        
        # TO GET PHASE - will assume np.nan before first ov or mens date
        
        if dates[i] in mens_dates_t or dates[i] in mens_buffer_dates:
            phase = 1

        elif dates[i] in ov_dates_t or dates[i] in ov_buffer_dates:
            phase = 3
        
        elif phase != np.nan:
            if phase == 1: # if not in mens dates or buffer dates but phase is currently 1, phase moves to follicular
                phase = 2
            elif phase == 3: # if not in ov dates or buffer dates but currently 2, moves to luteal
                phase = 4
            
        days_since[i] = days_since_count
        phase_list[i] = phase
        
    # add to df        
    df['days since'] = days_since
    df['phase'] = phase_list

In [51]:
for df in dfs:
    days_since_mens(df, df_dates)

In [52]:
def uva_cleaning(df):
    '''returns df cleaned with basal delivery/cgm change from cleaned/compiled input'''
    
    # CGM
    # generating rate of change, adding to col
    rate_list = np.zeros(len(df))

    for i in range(len(df)):
        curr_bg = (df.iloc[i]['cgm'])
        last_bg = (df.iloc[i-1]['cgm'])
        rate_list[i] = (curr_bg - last_bg) / 5

    rate_list[0] = np.nan 
    cgmdf = df[['t', 'cgm']].copy()
    cgmdf['cgm change'] = rate_list

    # BASAL
    # starts from rate / hour, converts to rate / 5 mins 
    
    # may need to redo - nan will be same rate, 0 will be a true 0 
    cond_basaldf = df[df['basal'].notnull()]
    cond_basaldf = cond_basaldf[['t', 'basal']]

    basal_cols = ['t', 'basal']
    basal_data = []
    
    for i in range(len(cond_basaldf)):
        # for each basal rate: get duration, delivered, delivery type, 
        # divide up into u delivered per timestamp
        start_time = cond_basaldf.iloc[i]['t']
        
        if i == len(cond_basaldf) - 1:
            end_time = df.iloc[-1]['t']
        else:
            end_time = cond_basaldf.iloc[i+1]['t']
        duration = (end_time - start_time).total_seconds() / 60
        intervals = int(duration // 5)

        units_5min = cond_basaldf.iloc[i]['basal'] / 12
        
        for j in range(intervals):
            # making all timestamps have basal delivered
            new_time = start_time + (j * (pd.Timedelta(minutes=5)))
            row = [new_time, units_5min]
            basal_data.append(row)
    
    if basal_data == []: # if empty, copy over np.nan from orignal df
        basal_data = df[['t', 'basal']]
        
    basaldf = pd.DataFrame(basal_data)
    basaldf.columns = basal_cols
    
    # BOLUS - no expansion needed for this data, just need to have to combine
    bolusdf = df[['t', 'bolus']]
            
    fooddf = df[['t', 'meals']]
    
    mensdf = df[['id', 't', 'date', 'time', 'is mens', 'is ov', 'days since', 'phase']]
    
    # combining
    insulindf = pd.merge(basaldf, bolusdf, how ='outer', on = 't')
    insulindf = insulindf.sort_values(by=['t'], ascending=True)
    intakedf = pd.merge(insulindf, fooddf, how ='outer', on = 't')
    mens_intakedf = pd.merge(intakedf, mensdf, how ='outer', on = 't')
    finaldf = pd.merge(cgmdf, mens_intakedf, how = 'outer', on = 't')
    
    return finaldf

In [53]:
cleaned_dfs = []
for i in range(len(dfs)):
    cleaned_dfs.append(uva_cleaning(dfs[i]))

In [25]:
# saving cleaned dfs as csvs
for df in cleaned_dfs:
    subj_id = str(df.iloc[0]['id'])
    csv_name = 'cleaned_dfs/' + subj_id + '.csv'
    df.to_csv(csv_name)

In [54]:
def get_baseline_basal(df, time_divs):
    '''returns average basal per 5 minutes across different times'''
    
    # dividing dataframe into time blocks for delivery
    time_chunk1 = df[(df['time'] >= time_divs[0]) & (df['time'] < time_divs[1])]
    time_chunk2 = df[(df['time'] >= time_divs[1]) & (df['time'] < time_divs[2])]
    time_chunk3 = df[(df['time'] >= time_divs[2]) & (df['time'] < time_divs[3])]
    time_chunk4 = df[(df['time'] >= time_divs[3])]
    
    # getting average basal delivered per 5 min per time block
    avg_basal1 = time_chunk1['basal'].mean()
    avg_basal2 = time_chunk2['basal'].mean()
    avg_basal3 = time_chunk3['basal'].mean()
    avg_basal4 = time_chunk4['basal'].mean()
    
    subj_id = df.iloc[0]['id']
    
    avg_schedule = [subj_id, avg_basal1, avg_basal2, avg_basal3, avg_basal4]

    return avg_schedule

In [64]:
# getting average basal delivery rates
basal_list = []

time_divs = ['00:00:00', '06:00:00', '12:00:00', '18:00:00']
for df in cleaned_dfs:
    basal_list.append(get_baseline_basal(df, time_divs))
    
basal_schedules = pd.DataFrame(basal_list)
basal_schedules.columns = ['id', 'time1', 'time2', 'time3', 'time4']

In [ ]:
def add_scheduled_basal(df):
    '''adds scheduled basal col to cleaned df'''

    time_conds = [df['time'] <= ('06:00:00'), df['time'] <= ('12:00:00'), df['time'] <= ('18:00:00'), df['time'] <= ('24:00:00')]
    time_choices = ['time1', 'time2', 'time3', 'time4']
        
    time_arr = np.select(time_conds, time_choices, default=0)
        
    subj_id = df.iloc[0]['id']
    basal_schedule = basal_schedules[basal_schedules['id'] == subj_id].iloc[0].to_numpy()
    
    basal_conds = [time_arr == 'time1', time_arr == 'time2', time_arr == 'time3', time_arr == 'time4']
    basal_choices = [basal_schedule[1], basal_schedule[2], basal_schedule[3], basal_schedule[4]]

    basal_arr = np.select(basal_conds, basal_choices, default=0)
    
    df['sched_basal'] = basal_arr

In [ ]:
for df in cleaned_dfs:
    add_scheduled_basal(df)

# base analysis

In [55]:
def bgi(df):
    '''returns lbgi/hbgi and scaled lbgi/hgbi (risk values)'''
    df = df.copy()
    df['scaled'] = (1.509*(np.log(df['cgm'])**1.084-5.381)).astype(float)
    df['risk_val'] = 10 *(df['scaled'] ** 2)
    df = df[df['risk_val'].notnull()].copy()
    
    if len(df) == 0:
        return [np.nan, np.nan, np.nan, np.nan]
    else:
        lbgi = len(df[df['scaled'] < 1.0]) / len(df)
        lbgi2 = np.nansum((df[df['scaled'] < 1.0])['risk_val']) / len(df)
        hbgi = len(df[df['scaled'] > 1.0]) / len(df)
        hbgi2 = np.nansum((df[df['scaled'] > 1.0])['risk_val']) / len(df)
    
    return [lbgi, lbgi2, hbgi, hbgi2]

In [56]:
def cgm_stats(df):
    ''' returns TIR, time<70 (tbr), time>180 (tar), time>250 (tvar), mean, stddev, and CV '''
    # time in range stats
    perc_coverage = len(df[df['cgm'].notnull()]) / len(df['cgm'])
    tot_data = len(df[df['cgm'].notnull()])
    if tot_data != 0 :
        tir = len(df[ (df['cgm'] > 70) & (df['cgm'] < 180)]) / len(df[df['cgm'].notnull()])
        tbr = len(df[ (df['cgm'] < 70)]) / len(df[df['cgm'].notnull()])
        tar = len(df[ (df['cgm'] > 180)]) / len(df[df['cgm'].notnull()])
        tvar = len(df[ (df['cgm'] > 250)]) / len(df[df['cgm'].notnull()])
    else:
        tir, tbr, tar, tvar = np.nan, np.nan, np.nan, np.nan
    
    # other statistics
    mean = np.nanmean(df['cgm'])
    stddev = df['cgm'].std()
    cv = (stddev / mean) * 100
    gmi = 3.31 + (0.02392 * mean)
    lbgi, lbgi2, hbgi, hbgi2 = bgi(df)
    
    results = [perc_coverage, tir, tbr, tar, tvar, mean, stddev, cv, gmi, lbgi, lbgi2, hbgi, hbgi2]
    
    return results

In [57]:
def carb_insulin_stats(df, time_div):
    ''' returns total insulin, total bolus, total basal, total carbs'''
    total_ins = df['bolus'].sum() + df['basal'].sum()
    total_bolus = df['bolus'].sum()
    total_basal = df['basal'].sum()
    total_carb = df['meals'].sum()
    
    if time_div == 'hours':
        start_time = df.iloc[0]['time']
        conditions = [start_time <= ('00:06:00'), start_time <= ('12:00:00'), start_time <= ('18:00:00'), start_time <= ('24:00:00')]
        choices = ['time1', 'time2', 'time3', 'time4']
        
        time_block = np.select(conditions, choices, default=0)
        
        subj_id = df.iloc[0]['id']
        scheduled_5min = basal_schedules[basal_schedules['id'] == subj_id].iloc[0][time_block]
        sched_basal = scheduled_5min * 12
    
    else:
        sched_basal = np.nan # FIX
    
    results = [total_ins, total_bolus, total_basal, sched_basal, total_carb]
    return results

In [58]:
def stats_df_row(df, time_div):
    '''returns combined cgm/insulin/carb stats with a datestamp to append to df'''
    row = []
    ind = len(df)
    row.append(df.iloc[0]['date'])
    row.append(df.iloc[0]['time'])
    row.append(df.iloc[0]['id'])
    row.append(df.iloc[0]['days since'])
    row.append(df.iloc[0]['is mens'])
    row.append(df.iloc[0]['is ov'])
    row.append(df.iloc[0]['phase'])
    row = row + cgm_stats(df)
    row = row + carb_insulin_stats(df, time_div)
    return row

In [59]:
def stats_df(df, time_div):
    '''returns dataframe of standard statistics from cleaned dataframe'''
    # using day or hour divs for now, will want different stats for 5 min intervals
    
    stats = []
    # day
    if time_div == 'days':
        data_dates = df['date'].unique()
        
        for day in data_dates:
            datedf = df[df['date']== day]
            stats.append(stats_df_row(datedf, time_div))
        
    # hour
    elif time_div == 'hours':
        df['day_hour'] = df['t'].dt.strftime('%m/%d/%Y %H')

        data_hours = df['day_hour'].unique()
        for hour in data_hours:
            hourdf = df[df['day_hour'] == hour]
            stats.append(stats_df_row(hourdf, time_div))
        
    cols = ['day', 'hour', 'id', 'days_since', 'is mens', 'is ov', 'phase', 'perc_coverage', 'TIR', 'TBR', 'TAR', 'TVAR', 'mean', 'stddev', 'cv', 'GMI',
            'lbgi', 'lbgi2', 'hbgi', 'hbgi2', 'total_ins', 'total_bolus', 'total_basal', 'sched_basal', 'total_carb']
    statsdf = pd.DataFrame(stats)
    statsdf.columns = cols
    
    return statsdf

In [60]:
cgm_stats_list = []

for df in dfs:
    cgm_stats_list.append(cgm_stats(df))

In [114]:
# list of stats from cleaned dataframes based on day or time divisions
stats_day_dfs = []
for df in cleaned_dfs:
    stats_day_dfs.append(stats_df(df, 'days')) 

/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(df['cgm'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(df['cgm'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(df['cgm'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(df['cgm'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(df['cgm'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(df['cgm'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: M

In [117]:
for df in stats_day_dfs:
    subj_id = str(df.iloc[0]['id'])
    csv_name = 'stats_dfs/day_updated/' + subj_id + '.csv'
    df.to_csv(csv_name)

In [118]:
# list of stats from cleaned dataframes based on day or time divisions
stats_hour_dfs = []
for df in cleaned_dfs:
    stats_hour_dfs.append(stats_df(df, 'hours')) 

/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(df['cgm'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(df['cgm'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(df['cgm'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(df['cgm'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(df['cgm'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(df['cgm'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/3663046464.py:15: RuntimeWarning: M

In [120]:
for df in stats_hour_dfs:
    subj_id = str(df.iloc[0]['id'])
    csv_name = 'stats_dfs/hour_updated/' + subj_id + '.csv'
    df.to_csv(csv_name)

# Block transformation

### Fine grained
CGM: 1-5 for 55, 70, 180, 250
Insulin: 1 if below standard, 2 if standard, 3 if above standard for both bolus/basal
- for basal: generate an "average" by getting avg delivery for each time - should be able to just use groupby to get this
- dividing to 12/6 hour segments for average basal delivery? 12-6 overnight - can try multiple methods
- for bolus: need an average bolus , 1 - 2 - 3 for within 20% of avg? need to reevaluate for meals probably or just 1/0
- food 1/0

In [61]:
cleaned_dfs[0].columns

Index(['t', 'cgm', 'cgm change', 'basal', 'bolus', 'meals', 'id', 'date',
       'time', 'is mens', 'is ov', 'days since', 'phase'],
      dtype='object')

In [67]:
for df in cleaned_dfs:
    df['cgm'].fillna(0)
    df['cgm change'].fillna(-1000)

In [94]:
def hashfxn_fine(df):
    '''more fleshed out hash function with some choices based on significant clinical values/goals'''
    vals = ['cgm', 'cgm change', 'basal', 'bolus', 'meals']
    
    hash_list = np.zeros(len(df)).astype(str)
    first_var = True
    
    for var in vals:
        # selecting conditions / choices
        if var == 'cgm': # basing off of euglycemic goals, 0 if no data
            conditions = [df[var] == 0, df[var] <= (55), df[var] <= (70), df[var] <= (180), df[var] <= 250, 250 < df[var]]
            choices = [0, 1, 2, 3, 4, 5]
        elif var == 'cgm change': # <-3, <-1, in betwen -1 and 1, > 1, > 3, 0 if no data
            conditions = [df[var] == -1000, df[var] <= (-3), df[var] <= (-1), df[var] <= (1), df[var] <= 3, 3 < df[var]]
            choices = [0, 1, 2, 3, 4, 5]
        elif var == 'basal': # within -30, -15, +15, +30 mean basal rate 
            conditions = [df[var] <= (.7 * df['sched_basal']), df[var] <= (.85 * df['sched_basal']), df[var] <= (1.15 * df['sched_basal']), 
                          df[var] <= 1.3 * df['sched_basal'], 1.3 * df['sched_basal'] < df[var]]
            choices = [1, 2, 3, 4, 5]  
                
            
        else: # if within -30, -15, +15, +30% of the mean value for the var (bolus and carbs)
            mean = df[var].mean()
            conditions = [df[var] == 0, df[var] <= (mean*.70), df[var] <= (mean*.85),df[var] <= mean*1.15,
                df[var] <= mean*1.30, mean*1.30 < df[var]]
            choices = [0, 1, 2, 3, 4, 5]

        # adding vals to list   
        arr = np.select(conditions, choices, default=0)
    
        if first_var == True:
            for i in range(len(hash_list)):
                hash_list[i] = arr[i].astype(str)
            first_var = False
        else:
            for i in range(len(hash_list)):
                hash_list[i] = str(hash_list[i].astype(str) + arr[i].astype(str))

    df['hash'] = hash_list

In [95]:
for df in cleaned_dfs:
    hashfxn_fine(df)

### Coarse grained
mean CGM value, tir, tbr, tar - based on clinical goals\
bolus - based on avg bolus \
basal - based on sched basal\
meals - based on avg meal \
if cgm data is missing - 0 if missing data, 1 if not \

In [46]:
def hashfxn_coarse(df):
    '''more fleshed out hash function with some choices based on significant clinical values/goals'''
    # avail cols: 'TIR', 'TBR', 'TAR', 'TVAR', 'perc_coverage 'mean', 'stddev', 'cv', 'GMI', 'lbgi', 'hbgi', 
    # 'incl_low', 'incl_high', 'incl_vhigh', 'total_ins', 'bolus_perc', 'basal_perc'
    vals = ['TIR', 'TBR', 'TAR', 'TVAR', 'mean', 'total_bolus', 'total_basal', 'total_carb', 'perc_coverage']
    
    hash_list = np.zeros(len(df)).astype(str)
    first_var = True
    
    for var in vals:
        # selecting conditions / choices
        if var == 'TIR': # goal is >.70, 3 = .65 to .75
            conditions = [df[var] <= (.55), df[var] <= (.65), df[var] <= (.75), df[var] <= (.85), .85 < df[var]]
            choices = [1, 2, 3, 4, 5]
        elif var == 'TBR': # goal is <.05, 3 = .04 - .06, 0 if low bg not seen
            conditions = [df[var] == 0, df[var] <= (.02), df[var] <= (.04), df[var] <= (.06), df[var] <= .08, .08 < df[var]]
            choices = [0, 1, 2, 3, 4, 5]
        elif var == 'TAR': # goal is < .25, 3 = .2 - .3, 0 if high bg not seen
            conditions = [df[var] == 0, df[var] <= (.1), df[var] <= (.2), df[var] <= .3, df[var] <= .4, .4 < df[var]]
            choices = [0, 1, 2, 3, 4, 5]
        elif var == 'TVAR': # goal is < .05, 3 = .04 - .06
            conditions = [df[var] == 0, df[var] <= (.02), df[var] <= (.04), df[var] <= (.06), df[var] <= .08, .08 < df[var]]
            choices = [0, 1, 2, 3, 4, 5]
        elif var == 'mean': # basing off of euglycemic goals
            conditions = [df[var] <= (55), df[var] <= (70), df[var] <= (180), df[var] <= 250, 250 < df[var]]
            choices = [1, 2, 3, 4, 5]
        elif var == 'total_basal': # within -30, -15, +15, +30 mean basal rate 
            conditions = [df[var] <= (.7 * df['sched_basal']), df[var] <= (.85 * df['sched_basal']), df[var] <= (1.15 * df['sched_basal']), 
                          df[var] <= 1.3 * df['sched_basal'], 1.3 * df['sched_basal'] < df[var]]
            choices = [1, 2, 3, 4, 5]
        elif var == 'perc_coverage': # 1 if not missing CGM data, 0 if missing found
            conditions = [df[var] < 1, df[var] == 1]
            choices = [0, 1]
                
            
        else: # if within -30, -15, +15, +30% of the mean value for the var (bolus and carbs)
            mean = df[var].mean()
            conditions = [df[var] == 0, df[var] <= (mean*.70), df[var] <= (mean*.85),df[var] <= mean*1.15,
                df[var] <= mean*1.30, mean*1.30 < df[var]]
            choices = [0, 1, 2, 3, 4, 5]

        # adding vals to list   
        arr = np.select(conditions, choices, default=0)
    
        if first_var == True:
            for i in range(len(hash_list)):
                hash_list[i] = arr[i].astype(str)
            first_var = False
        else:
            for i in range(len(hash_list)):
                hash_list[i] = hash_list[i].astype(str) + arr[i].astype(str)

    df['hash'] = hash_list

In [47]:
for df in stats_hour_dfs:
    hashfxn_coarse(df)

In [48]:
for df in stats_hour_dfs:
    subj_id = str(df.iloc[0]['id'])
    csv_name = 'coarse/hashes_updated/' + subj_id + '_coarse.csv'
    col = ['hash']
    df.to_csv(csv_name, columns= col)

In [49]:
coarse_df_id_order = []
for df in stats_hour_dfs:
    coarse_df_id_order.append(str(df.iloc[0]['id']))

In [98]:
for df in cleaned_dfs:
    subj_id = str(df.iloc[0]['id'])
    csv_name = 'fine/hashes_updated/' + subj_id + '_fine.csv'
    col = ['hash']
    df.to_csv(csv_name, columns= col)

In [51]:
fine_df_id_order = []
for df in cleaned_dfs:
    fine_df_id_order.append(str(df.iloc[0]['id']))

# Matrix creation
This is outdated - matrix creation now completed in matrix_gen.ipynb for efficiency

In [52]:
def get_matrix(df, var):
    '''returns binary matrix for similarity from stats df and var (typically 'hash')'''
    rows, cols = (len(df), len(df))
    arr = [[0 for i in range(cols)] for j in range(rows)]
        
    for i in range(len(df)):
        for j in range(len(df)):
            if (df.iloc[i][var] == df.iloc[j][var]):
                # if equal, returns 1
                arr[i][j] = 1
    return arr

# THIS FXN NO LONGER USED

In [100]:
def build_RQA_plot(df, var):
    a = df[var].values.flatten()
    N = len(a)
    recMat = np.eye(N)
    
    for j in range(1,N):
        np.fill_diagonal(recMat[j:,:-j], np.equal(a[j:],a[:-j]))

    recMat = recMat + np.transpose(recMat)

    recMat = recMat - np.eye(N)

    return recMat
    
# from Taisa 

In [101]:
fine_mat_list = []
for df in cleaned_dfs:
    id = df.iloc[0]['id']
    fine_mat = build_RQA_plot(df, 'hash')
    

: 

In [ ]:
coarse_mat_list = []
for df in stats_hour_dfs:
    coarse_mat_list.append(build_RQA_plot(df, 'hash'))

# Run finding

In [63]:
def get_runs(df, arr, var):
    '''returns lists of start and end coordinates of pattern runs from df, arr, and var name'''         
    # traverse diags, get arr of list of diag values
    diags = []
    for diag in range(1, len(arr)): 
        i = 0
        j = diag
        diagonal = []
        while j < len(arr): 
            diagonal.append(arr[i][j])
            i += 1
            j += 1
        diags.append(diagonal)
        
    run_df_cols = ['start_coord', 'end_coord', 'all_coords', 'all_hashes', 'length', 'is_edge']
    run_data = []
    
    # for each diagonal: get series of diffs (+1 = start, -1 = end)
    for i in range(len(diags)):
        diffs = pd.Series(diags[i]).diff()
        closed = True # flag for if run hits end of diag
        start_pos = 0 # if starts on 1
        edge = False
        #print(i)
        for j in range(len(diffs)): 
            if (diffs[j] == 1) or (j == 0 and diags[i][0] == 1): # start points
                run_start = [i + j + 1, j]
                start_pos = j
                closed = False
                # initializes lists for storing run values (coords and hashes)
                coords = [run_start]
                hashes = [df.iloc[j][var]]
                # edge case: if the first value in a diag is 1
                if (j == 0 & diags[i][0] == 1):
                        edge = True
            elif (diffs[j] == -1) or ((j == len(diffs)-1) & (closed == False)): # end points
                run_end = [i + j, j - 1]
                end_pos = j
                length = (end_pos - start_pos)
                closed = True
                # edge case: if last value in a diag is 1 
                if j == len(diffs)-1:
                        edge = True
                # row has start coord, end coord, list of coordinates, list of values, length, and if edge  
                row = [run_start, run_end, coords, hashes, length, edge]
                run_data.append(row)
            elif (diffs[j] == 0) & (closed == False): # part of run, adds coord and hash to lists
                coords.append([i + j + 1, j])
                hashes.append(df.iloc[j][var])

    run_df = pd.DataFrame(run_data)
    run_df.columns = run_df_cols
    return run_df

In [64]:
def sort_repeats(run_df, patt):
    '''returns individual starts of pattern, removes overlaps'''
    all_coords_list = (run_df[run_df['all_hashes'].astype(str) == patt])['all_coords']
    run_locs = []
    for diag in all_coords_list:
        run_locs.append([coord[0] for coord in diag])
        run_locs.append([coord[1] for coord in diag])
    
    # remove exact repeats
    res = []
    [res.append(x) for x in run_locs if x not in res]

    # remove overlaps
    unique_values = set()
    remaining = []
    for sublist in res:
        has_overlap = False
        for value in sublist:
            if value in unique_values:
                has_overlap = True
                break
            unique_values.add(value)
        if not has_overlap:
            remaining.append(sublist)
    return remaining

In [65]:
def get_patt_df(run_df):
    '''returns df of patterns, length, repeats, start coords'''
    patt_list = run_df['all_hashes'].astype(str).unique()
    # get list of unique patterns

    patt_cols = ['pattern', 'length', 'repeats', 'unique_blocks', 'ind_runs', 
                 'start_coords', 'incl_low', 'incl_high', 'incl_food']
    patt_data = []
    for patt in patt_list:
        length = len((patt.strip('][').split(', ')))
        # getting pattern length, getting rid of overlaps & taking first occurence if overlap
        ind_starts = sort_repeats(run_df, patt)
        repeats = len(ind_starts)
        if repeats == 1:
            continue
            # do not add to pattern df if only repeated once 
        # number of times pattern seen
        unique = np.unique(patt.strip('][').split(', '))
        # unique blocks, strip since lists are stored as strings
        all_starts = run_df[run_df['all_hashes'].astype(str) == patt]['start_coord'].to_numpy()
        # array of start coords - not cute but it works
        
        incl_low = False
        incl_high = False
        incl_food = False
        for block in unique: # if low/high in any block, include
            if block[2] != '0':
                incl_low = True
            if block[3] != '0':
                incl_high = True
            if block[-3] != '0':
                incl_food = True
        # low/high included in pattern
                
        patt_data.append([patt, length, repeats, unique, ind_starts, all_starts, incl_low, incl_high, incl_food])
        
    patt_df = pd.DataFrame(patt_data)
    patt_df.columns = patt_cols
    return patt_df

### Coarse run/pattern dfs

In [78]:
coarse_run_dfs = []
coarse_patt_dfs = []
var = 'hash'

for df in stats_hour_dfs:

    subj_id = str(df.iloc[0]['id'])
    
    file_name = 'coarse/matrices_updated/' + subj_id + '_coarse_mat.csv'
    with open(file_name, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)

    mat = np.array(data)
    mat[mat=='']='0'
    mat = mat.astype(float)
    mat = mat.astype(int)

    rundf = get_runs(df, mat, var)
    coarse_run_dfs.append(rundf)
    pattdf = get_patt_df(rundf)
    coarse_patt_dfs.append(pattdf)

In [79]:
for i in range(len(coarse_run_dfs)):
    subj_id = coarse_df_id_order[i]
    df = coarse_run_dfs[i]
    csv_name = 'coarse/run_dfs_updated/' + subj_id + '_coarse.csv'
    df.to_csv(csv_name)

In [80]:
for i in range(len(coarse_patt_dfs)):
    subj_id = coarse_df_id_order[i]
    df = coarse_patt_dfs[i]
    csv_name = 'coarse/patt_dfs_updated/' + subj_id + '_coarse.csv'
    df.to_csv(csv_name)

### Fine run/pattern dfs
not used - memory issues

In [ ]:
fine_run_dfs = []
fine_patt_dfs = []
var = 'hash'

for df in cleaned_dfs:

    subj_id = str(df.iloc[0]['id'])
    print(subj_id)
    
    file_name = 'fine/results/' + subj_id + '_fine_mat.csv'
    with open(file_name, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)

    mat = np.array(data)
    mat[mat=='']='0'
    mat = mat.astype(float)
    mat = mat.astype(int)

    rundf = get_runs(df, mat, var)
    fine_run_dfs.append(rundf)
    pattdf = get_patt_df(rundf)
    fine_patt_dfs.append(pattdf)


78132


: 

# Analysis functions, getting outcome dataframes (split into time and phases)

In [121]:
def post_df_get_stats(post_df, unique_outcomes):
    '''returns outcome stats from df of 2hour times post pattern'''
    
    if len(post_df) > 0:
        low_count = len(post_df[post_df['TBR']>0])
        low_perc = low_count / (unique_outcomes * 2)
        high_count = len(post_df[post_df['TAR']>0])
        high_perc = high_count / (unique_outcomes * 2)
        vhigh_count = len(post_df[post_df['TVAR']>0])
        vhigh_perc = vhigh_count / (unique_outcomes * 2)

        avg_cgm = np.nanmean(post_df['mean'])
        avg_lbgi = np.nanmean(post_df['lbgi'])
        avg_lbgi_risk = np.nanmean(post_df['lbgi2'])
        avg_hbgi = np.nanmean(post_df['hbgi'])
        avg_hbgi_risk = np.nanmean(post_df['hbgi2'])
        avg_tir = np.nanmean(post_df['TIR'])
        avg_tbr = np.nanmean(post_df['TBR'])
        avg_tar = np.nanmean(post_df['TAR'])
        avg_tvar = np.nanmean(post_df['TVAR'])

    elif len(post_df) == 0:
        return [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 
                np.nan, np.nan,np.nan, np.nan]
    
    return [low_count, low_perc, high_count, high_perc, vhigh_count, vhigh_perc, avg_cgm, avg_lbgi, avg_lbgi_risk,
            avg_hbgi, avg_hbgi_risk, avg_tir, avg_tbr, avg_tar, avg_tvar]

In [122]:
def patt_outcomes(pattdf, statsdf):
    ''' returns outcomes for the 2 hours after a pattern is seen and adds to pattdf'''
    full_patt_cols = ['pattern', 'length', 'repeats', 'unique_blocks', 'ind_runs', 'start_coords', 
                      'incl_low', 'incl_high', 'incl_food',
                      'unique_outcomes', 'led_high', 'led_low', 'low_count', 'low_perc',
                      'high_count', 'high_perc', 'vhigh_count', 'vhigh_perc', 'avg_cgm', 
                      'avg_lbgi', 'avg_lbgi_risk', 'avg_hbgi', 'avg_hbgi_risk', 'avg_tir', 'avg_tbr', 'avg_tar', 'avg_tvar']
    full_patt_data = []
    
    for i in range(len(pattdf)):
        post_df = pd.DataFrame(columns = statsdf.columns) 
        run_list = pattdf.iloc[i]['ind_runs']
        length = pattdf.iloc[i]['length']
        first_add = True
        led_high = []
        led_low = []
        
        for run in run_list:
            # adding 2 hour segments post pattern to df
            start = run[-1] + 1 # last location + 1 
            slice_df = statsdf.iloc[start:start+2]
            if len(slice_df[slice_df['TAR'] > 0]) > 0 :
                led_high.append([start, start + 1])
            if len(slice_df[slice_df['TBR'] > 0]):
                led_low.append([start, start + 1])
            
            if first_add == True:
                first_add = False
                post_df = slice_df
                
            else:
                post_df = pd.concat([post_df, slice_df])
                
        post_df = post_df.drop_duplicates()
        
        # post stats: divided into includes or other statistics, done in post_df_get_stats fxn
        unique_outcomes = len(run_list)

        patt_aslist = pattdf.loc[i, :].values.flatten().tolist()
        stats_aslist = [unique_outcomes, led_low, led_high]
        
        for stat in stats_aslist:
            patt_aslist.append(stat)
           
        full_patt_row = patt_aslist
        
        stats2_aslist = post_df_get_stats(post_df, unique_outcomes)
        for stat in stats2_aslist:
            full_patt_row.append(stat)

        full_patt_data.append(full_patt_row)

    full_patt_df = pd.DataFrame(full_patt_data)
    full_patt_df.columns = full_patt_cols

    
    return full_patt_df

In [ ]:
def get_time_div_df(pattdf, statsdf, time_divs):
    '''basically same as outcome df but split by times'''
    time_div_cols = ['pattern', 'length', 'repeats', 'unique_blocks', 'ind_runs', 'start_coords', 
                     'incl_low', 'incl_high', 'incl_food', 
                     'time1_runs', 'time2_runs', 'time3_runs', 'time4_runs', 
                     'unique_outcomes', 'led_high', 'led_low',
                     't1_low_count', 't1_low_perc','t1_high_count', 't1_high_perc', 't1_vhigh_count',
                     't1_vhigh_perc', 't1_avg_cgm', 't1_avg_lbgi', 't1_avg_lbgi_risk', 't1_avg_hbgi', 't1_avg_hbgi_risk'
                     't1_avg_tir', 't1_avg_tbr', 't1_avg_tar', 't1_avg_tvar',
                     't2_low_count', 't2_low_perc','t2_high_count', 't2_high_perc', 't2_vhigh_count', 
                     't2_vhigh_perc', 't2_avg_cgm', 't2_avg_lbgi', 't2_avg_lbgi_risk', 't2_avg_hbgi', 't2_avg_hbgi_risk', 
                     't2_avg_tir', 't2_avg_tbr', 't2_avg_tar', 't2_avg_tvar',
                     't3_low_count', 't3_low_perc','t3_high_count', 't3_high_perc', 't3_vhigh_count', 
                     't3_vhigh_perc', 't3_avg_cgm', 't3_avg_lbgi', 't3_avg_hbgi', 
                     't3_avg_tir', 't3_avg_tbr', 't3_avg_lbgi', 't3_avg_lbgi_risk', 't3_avg_hbgi', 't3_avg_hbgi_risk',
                     't4_low_count', 't4_low_perc','t4_high_count', 't4_high_perc', 't4_vhigh_count', 
                     't4_vhigh_perc', 't4_avg_cgm', 't4_avg_lbgi', 't4_avg_lbgi_risk', 't4_avg_hbgi', 't4_avg_hbgi_risk',
                     't4_avg_tir', 't4_avg_tbr', 't4_avg_tar', 't4_avg_tvar']
    time_div_data = []
      
    inds1_1 = statsdf[(statsdf['hour'] >= '00:00:00') & (statsdf['hour'] < time_divs[0])].index.values
    inds1_2 = statsdf[(statsdf['hour'] >= time_divs[3]) & (statsdf['hour'] < '24:00:00')].index.values
    time_inds1 = np.append(inds1_1, inds1_2)
    time_inds2 = statsdf[(statsdf['hour'] >= time_divs[0]) & (statsdf['hour'] < time_divs[1])].index.values
    time_inds3 = statsdf[(statsdf['hour'] >= time_divs[1]) & (statsdf['hour'] < time_divs[2])].index.values
    time_inds4 = statsdf[(statsdf['hour'] >= time_divs[2]) & (statsdf['hour'] < time_divs[3])].index.values
    
    for i in range(len(pattdf)):
        post_df = pd.DataFrame(columns = statsdf.columns) 
        run_list = pattdf.iloc[i]['ind_runs']
        length = pattdf.iloc[i]['length']
        first_add = True
        led_high = []
        led_low = []
        time1 = []
        time2 = []
        time3 = []
        time4 = []
        
        time1add1, time2add1, time3add1, time4add1 = True, True, True, True
        
        for run in run_list:
            # adding 2 hour segments post pattern to df
            start = run[-1] + 1 # last location + 1 
            slice_df = statsdf.iloc[start:start+2]
            if len(slice_df[slice_df['TAR'] > 0]) > 0:
                led_high.append([start, start + 1])
            if len(slice_df[slice_df['TBR'] > 0]) >0 :
                led_low.append([start, start + 1])
                
            # adding start ind to correct timeframe
            time1post = pd.DataFrame(columns = slice_df.columns)
            time2post = pd.DataFrame(columns = slice_df.columns)
            time3post = pd.DataFrame(columns = slice_df.columns)
            time4post = pd.DataFrame(columns = slice_df.columns)
            
            if run[0] in time_inds1:
                time1.append(run)
                if time1add1 == True:
                    time1post = slice_df
                    time1add1 = False
                else:
                    time1post = pd.concat([time1post, slice_df])
            elif run[0] in time_inds2:
                time2.append(run)
                if time2add1 == True:
                    time2post = slice_df
                    time2add1 = False
                else:
                    time2post = pd.concat([time2post, slice_df])
            elif run[0] in time_inds3:
                time3.append(run)
                if time3add1 == True:
                    time3post = slice_df
                    time3add1 = False
                else:
                    time3post = pd.concat([time3post, slice_df])
            elif run[0] in time_inds4:
                time4.append(run)
                if time4add1 == True:
                    time4post = slice_df
                    time4add1 = False
                else:
                    time4post = pd.concat([time4post, slice_df])
                
            
        # post stats: divided into includes or other statistics, done in post_df_get_stats fxn
        unique_outcomes = len(run_list)

        patt_aslist = pattdf.loc[i, :].values.flatten().tolist()
        stats_aslist = [time1, time2, time3, time4, unique_outcomes, led_low, led_high]
        
        for stat in stats_aslist:
            patt_aslist.append(stat)
           
        time_div_row = patt_aslist.copy()
        
        time_posts = [time1post, time2post, time3post, time4post]
        time_runs = [time1, time2, time3, time4]
        
        for i in range(len(time_posts)):
            df = time_posts[i]
            outcome_num = len(time_runs[i])
            time_stats_aslist = post_df_get_stats(df, outcome_num)

            for stat in time_stats_aslist:
                time_div_row.append(stat)
        
        time_div_data.append(time_div_row)
    
    time_div_df = pd.DataFrame(time_div_data)
    time_div_df.columns = time_div_cols
            
    return time_div_df

In [138]:
def get_phase_div_df(pattdf, statsdf):
    '''basically same as outcome df but split by cycle phase'''
    phase_div_cols = ['pattern', 'length', 'repeats', 'unique_blocks', 'ind_runs', 'start_coords', 
                     'incl_low', 'incl_high', 'incl_food', 
                     'phase1_runs', 'phase2_runs', 'phase3_runs', 'phase4_runs', 
                     'unique_outcomes', 'led_high', 'led_low',
                     'p1_low_count', 'p1_low_perc','p1_high_count', 'p1_high_perc', 'p1_vhigh_count',
                     'p1_vhigh_perc', 'p1_avg_cgm', 'p1_avg_lbgi', 'p1_avg_lbgi_risk', 'p1_avg_hbgi', 'p1_avg_hbgi_risk',
                     'p1_avg_tir', 'p1_avg_tbr', 'p1_avg_tar', 'p1_avg_tvar',
                     'p2_low_count', 'p2_low_perc','p2_high_count', 'p2_high_perc', 'p2_vhigh_count', 
                     'p2_vhigh_perc', 'p2_avg_cgm', 'p2_avg_lbgi', 'p2_avg_lbgi_risk', 'p2_avg_hbgi', 'p2_avg_hbgi_risk',
                     'p2_avg_tir', 'p2_avg_tbr', 'p2_avg_tar', 'p2_avg_tvar',
                     'p3_low_count', 'p3_low_perc','p3_high_count', 'p3_high_perc', 'p3_vhigh_count', 
                     'p3_vhigh_perc', 'p3_avg_cgm', 'p3_avg_lbgi', 'p3_avg_lbgi_risk', 'p3_avg_hbgi', 'p3_avg_hbgi_risk',
                     'p3_avg_tir', 'p3_avg_tbr', 'p3_avg_tar', 'p3_avg_tvar',
                     'p4_low_count', 'p4_low_perc','p4_high_count', 'p4_high_perc', 'p4_vhigh_count', 
                     'p4_vhigh_perc', 'p4_avg_cgm', 'p4_avg_lbgi', 'p4_avg_lbgi_risk', 'p4_avg_hbgi', 'p4_avg_hbgi_risk',
                     'p4_avg_tir', 'p4_avg_tbr', 'p4_avg_tar', 'p4_avg_tvar']
    phase_div_data = []
    
    phase_inds1 = statsdf[statsdf['phase'] == 1].index.values.tolist()
    phase_inds2 = statsdf[statsdf['phase'] == 2].index.values.tolist()
    phase_inds3 = statsdf[statsdf['phase'] == 3].index.values.tolist()
    phase_inds4 = statsdf[statsdf['phase'] == 4].index.values.tolist()
    
    for i in range(len(pattdf)):
        run_list = pattdf.iloc[i]['ind_runs']
        length = pattdf.iloc[i]['length']
        led_high = []
        led_low = []
        phase1 = []
        phase2 = []
        phase3 = []
        phase4 = []
        
        phase1post = pd.DataFrame(columns = statsdf.columns)
        phase2post = pd.DataFrame(columns = statsdf.columns)
        phase3post = pd.DataFrame(columns = statsdf.columns)
        phase4post = pd.DataFrame(columns = statsdf.columns)
        
        phase1add1, phase2add1, phase3add1, phase4add1 = True, True, True, True
        
        for run in run_list:
            # adding 2 hour segments post pattern to df
            start = run[-1] + 1 # last location + 1 
            slice_df = statsdf.iloc[start:start+2]
            if len(slice_df[slice_df['TAR'] > 0]) > 0:
                led_high.append([start, start + 1])
            if len(slice_df[slice_df['TBR'] > 0]) > 0:
                led_low.append([start, start + 1])
                
            # adding start ind to correct timeframe
            if run[0] in phase_inds1:
                phase1.append(run)
                if phase1add1 == True:
                    phase1post = slice_df
                    phase1add1 = False
                else:
                    phase1post = pd.concat([phase1post, slice_df])
            elif run[0] in phase_inds2:
                phase2.append(run)
                if phase2add1 == True:
                    phase2post = slice_df
                    phase2add1 = False
                else:
                    phase2post = pd.concat([phase2post, slice_df])
            elif run[0] in phase_inds3:
                phase3.append(run)
                if phase3add1 == True:
                    phase3post = slice_df
                    phase3add1 = False
                else:
                    phase3post = pd.concat([phase3post, slice_df])
            elif run[0] in phase_inds4:
                phase4.append(run)
                if phase4add1 == True:
                    phase4post = slice_df
                    phase4add1 = False
                else:
                    phase4post = pd.concat([phase4post, slice_df])
                
        # post stats: divided into includes or other statistics, done in post_df_get_stats fxn
        unique_outcomes = len(run_list)

        patt_aslist = pattdf.loc[i, :].values.flatten().tolist()
        stats_aslist = [phase1, phase2, phase3, phase4, unique_outcomes, led_low, led_high]
        
        for stat in stats_aslist:
            patt_aslist.append(stat)
           
        phase_div_row = patt_aslist.copy()
        
        phase_posts = [phase1post, phase2post, phase3post, phase4post]
        phase_runs = [phase1, phase2, phase3, phase4]
        
        for i in range(len(phase_posts)):
            df = phase_posts[i]
            outcome_num = len(phase_runs[i])
            phase_stats_aslist = post_df_get_stats(df, outcome_num)

            for stat in phase_stats_aslist:
                phase_div_row.append(stat)
        
        phase_div_data.append(phase_div_row)
    
    phase_div_df = pd.DataFrame(phase_div_data)
    phase_div_df.columns = phase_div_cols
            
    return phase_div_df

In [124]:
def run_stats(mat, pattdf):
    '''returns %rec, %det, ADL, MDL'''
    rec = np.sum(mat) / (len(mat) ** 2)
    num_ones = (pattdf[pattdf['length'] == 1])['repeats'].sum()
    det = (np.sum(mat) - num_ones) / (np.sum(mat))
    adl = pattdf['length'].mean()
    mdl = pattdf['length'].max()
    
    return [rec, det, adl, mdl]

### Coarse analysis

In [129]:
#saving dfs as csv immediately instead of in list for memory's sake
for i in range(len(coarse_patt_dfs)):
    patt_df = coarse_patt_dfs[i]
    stat_df = stats_hour_dfs[i].reset_index(drop = True)
    outcome_df = patt_outcomes(patt_df, stat_df)
    
    subj_id = coarse_df_id_order[i]
    csv_name = 'coarse/outcome_dfs_updated/' + subj_id + '_coarse.csv'
    outcome_df.to_csv(csv_name)

/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/194859728.py:12: RuntimeWarning: Mean of empty slice
  avg_cgm = np.nanmean(post_df['mean'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/194859728.py:13: RuntimeWarning: Mean of empty slice
  avg_lbgi = np.nanmean(post_df['lbgi'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/194859728.py:14: RuntimeWarning: Mean of empty slice
  avg_lbgi_risk = np.nanmean(post_df['lbgi2'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/194859728.py:15: RuntimeWarning: Mean of empty slice
  avg_hbgi = np.nanmean(post_df['hbgi'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/194859728.py:16: RuntimeWarning: Mean of empty slice
  avg_hbgi_risk = np.nanmean(post_df['hbgi2'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/194859728.py:17: RuntimeWarning: Mean of empty slice
  avg_tir = np.nanmean(post_df['TIR'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6

In [ ]:
for i in range(len(coarse_patt_dfs)):
    patt_df = coarse_patt_dfs[i]
    stat_df = stats_hour_dfs[i].reset_index(drop = True)

    time_div_df = get_time_div_df(patt_df, stat_df, ['00:00:00', '06:00:00', '12:00:00', '18:00:00'])

    subj_id = coarse_df_id_order[i]
    csv_name = 'coarse/time_outcome_updated/' + subj_id + '_coarse.csv'
    time_div_df.to_csv(csv_name)

In [139]:
for i in range(len(coarse_patt_dfs)):
    patt_df = coarse_patt_dfs[i]
    stat_df = stats_hour_dfs[i].reset_index(drop = True)

    phase_div_df = get_phase_div_df(patt_df, stat_df)

    subj_id = coarse_df_id_order[i]
    csv_name = 'coarse/phase_outcome_updated/' + subj_id + '_coarse.csv'
    phase_div_df.to_csv(csv_name)

/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/194859728.py:12: RuntimeWarning: Mean of empty slice
  avg_cgm = np.nanmean(post_df['mean'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/194859728.py:13: RuntimeWarning: Mean of empty slice
  avg_lbgi = np.nanmean(post_df['lbgi'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/194859728.py:14: RuntimeWarning: Mean of empty slice
  avg_lbgi_risk = np.nanmean(post_df['lbgi2'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/194859728.py:15: RuntimeWarning: Mean of empty slice
  avg_hbgi = np.nanmean(post_df['hbgi'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/194859728.py:16: RuntimeWarning: Mean of empty slice
  avg_hbgi_risk = np.nanmean(post_df['hbgi2'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6vm40000gq/T/ipykernel_16156/194859728.py:17: RuntimeWarning: Mean of empty slice
  avg_tir = np.nanmean(post_df['TIR'])
/var/folders/bn/z7m_xmps3lxf8zqn27_l6